# Elise Backdoor
Detects Elise backdoor acitivty as used by APT32

## Rule Content
```
- title: Elise Backdoor
  id: e507feb7-5f73-4ef6-a970-91bb6f6d744f
  status: experimental
  description: Detects Elise backdoor acitivty as used by APT32
  references:
  - https://community.rsa.com/community/products/netwitness/blog/2018/02/13/lotus-blossom-continues-asean-targeting
  tags:
  - attack.g0030
  - attack.g0050
  - attack.s0081
  author: Florian Roth
  date: 2018/01/31
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection1:
      Image: C:\Windows\SysWOW64\cmd.exe
      CommandLine: '*\Windows\Caches\NavShExt.dll *'
    selection2:
      CommandLine: '*\AppData\Roaming\MICROS~1\Windows\Caches\NavShExt.dll,Setting'
    condition: 1 of them
  falsepositives:
  - Unknown
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path:"C\:\\Windows\\SysWOW64\\cmd.exe" AND process_command_line.keyword:*\\Windows\\Caches\\NavShExt.dll\ *) OR process_command_line.keyword:*\\AppData\\Roaming\\MICROS\~1\\Windows\\Caches\\NavShExt.dll,Setting)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()